In [ ]:
import cv2
import numpy as np
import scipy.ndimage
import scipy.stats
import time
import multiprocessing
import ImportNotebook
import GenerateDaugmanTemplateSet
import DaugmanIrisCore
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import copy as cp


mypool =  multiprocessing.Pool()
imageWorkingWidth = 150.0
irisRangeLB = 0.16
irisRangeUB = 0.24
irisTemplateSet = []
irisWeights = []
irisRange = []
irisRadiiLog = []
efficient=np.array([1,0.2,0.2,1.3])

def regulateIrisRange():
    from scipy.stats import norm
    (mu, sigma) = norm.fit(irisRadiiLog)
    global irisRangeLB
    global irisRangeUB
    irisRangeLB = mu - 2*sigma
    irisRangeUB = mu + 2*sigma
    generateTemplateSet()

def generateTemplateSet():
    global irisTemplateSet
    global irisWeights
    global irisRange
    irisRange = np.arange(round(irisRangeLB*imageWorkingWidth),round(irisRangeUB*imageWorkingWidth)+1,1)
    irisRingWidth = 1
    GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet.pool = mypool
    irisTemplateSet,irisWeights = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet(irisRingWidth,irisRange)

def generateTemplateSet2():
    global irisTemplateSet2
    global irisWeights2
    global irisRange2
    #强制制定模板大小
    irisRange2 = np.array([24.,25.,26.,27.,28.,29.,30.,31.,32.,33.,34.,35.])
    irisRingWidth2 = 1
    irisTemplateSet2,irisWeights2,index2 = GenerateDaugmanTemplateSet.GenerateDaugmanTemplateSet2(irisRingWidth2,irisRange2)
    
def EqualizeRGBImage(img):
    ycrcb = cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
    ycrcb[:,:,0] = cv2.equalizeHist(ycrcb[:,:,0])
    return cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2RGB)

#3.2图像预处理和仿射变换
def preprocessing(img,count,num):
    countStr='%d'%count
    numStr='%d'%num
    #cv2.imwrite('/Users/ly/Code/picBefore/eye'+countStr+'_'+numStr+'.jpg',img)
    #做了亮度的处理，为了更好的找到边界
    imgeq = EqualizeRGBImage(img)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/imgeq/eye'+countStr+'_'+numStr+'.jpg',cv2.cvtColor(imgeq,cv2.COLOR_RGB2BGR))
    #cv2.imshow('test1',img)
    imglarge = cv2.resize(imgeq,(np.int(imageWorkingWidth),np.int(np.ceil(imageWorkingWidth/imgeq.shape[1]*imgeq.shape[0]))),interpolation=cv2.INTER_LINEAR)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/imglarge/eye'+countStr+'_'+numStr+'.jpg',cv2.cvtColor(imglarge,cv2.COLOR_RGB2BGR))
    #cv2.imshow('test2',imglarge)
    #保护边缘的平滑滤波器bilateralFilter
    imgfiltered = cv2.bilateralFilter(imglarge,5,150,50)
    #cv2.imshow('test3',imgfiltered)
    #print imgfiltered.shape
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/eye/eye'+countStr+'_'+numStr+'.jpg',cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2BGR))
    imggrey = cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2GRAY)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/imggrey/eye'+countStr+'_'+numStr+'.jpg',imggrey)
    #cv2.imshow('test4',imggrey)
    return imggrey,imgfiltered

#3.4聚类
def fillHoleInBinary(Img,binaryImg):
    print '-------'
    print 'fillHoleInBinary'
    contours,hierarchy=cv2.findContours(binaryImg,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    if len(contours)!=0:
        for i in xrange(0,len(contours)):
            if len(contours[i])<40:
                cv2.drawContours(Img,contours,i,(40,40,40),-1)
    else:
        print 'no edge is detected!'
    return Img

In [ ]:
def UseKmeans(noHoleImg,Img,count,lor):
    print '-------'
    print 'UseKmeans'
    l1=np.reshape(noHoleImg[:,:,0].T,(Img.shape[0]*Img.shape[1],1))
    l2=np.reshape(noHoleImg[:,:,1].T,(Img.shape[0]*Img.shape[1],1))
    l3=np.reshape(noHoleImg[:,:,2].T,(Img.shape[0]*Img.shape[1],1))
    x=np.reshape(np.tile(np.arange(Img.shape[0]),Img.shape[1]),(l1.shape[0],1))
    y=np.reshape(np.arange(Img.shape[1]).repeat(Img.shape[0]),(l1.shape[0],1))
    data=np.hstack((l1*efficient[0],l2*efficient[1],l3*efficient[2],x*efficient[3],y*efficient[3]))
    kmeans=KMeans(n_clusters=2,random_state=0).fit(data)
    labels=kmeans.predict(data)
    if np.mean(data[np.where(labels==1),0:3])>np.mean(data[np.where(labels==0),0:3]):
        ti=0
    else:
        ti=1
    labels[np.where(labels!=ti)]=4
    newdata=data[np.where(labels==ti)]
    newdataIndex=np.where(labels==ti)[0]
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0:3])>np.mean(newdata[np.where(newlabels==0),0:3]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    labels[np.where(labels==0)]=3
    newdata=data[np.where(labels==1)]
    newdataIndex=np.where(labels==1)[0]
    kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    newlabels=kmeans.predict(newdata)
    if np.mean(newdata[np.where(newlabels==1),0])>np.mean(newdata[np.where(newlabels==0),0]):
        labels[newdataIndex[np.where(newlabels==1)[0]]]=0
        labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    else:
        labels[newdataIndex[np.where(newlabels==0)[0]]]=0
        labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    
    labels[np.where(labels!=1)]=0
    labels=np.uint8(np.reshape(labels,(Img.shape[1],Img.shape[0])).T*255)
    #cv2.imshow('labels',labels)
    
    #labels[np.where(labels==0)]=2
    #newdata=data[np.where(labels==1)]
    #newdataIndex=np.where(labels==1)[0]
    #kmeans=KMeans(n_clusters=2,random_state=0).fit(newdata)
    #newlabels=kmeans.predict(newdata)
    #if np.mean(newdata[np.where(newlabels==1),0])>np.mean(newdata[np.where(newlabels==0),0]):
    #    labels[newdataIndex[np.where(newlabels==1)[0]]]=0
    #    labels[newdataIndex[np.where(newlabels==0)[0]]]=1
    #else:
    #    labels[newdataIndex[np.where(newlabels==0)[0]]]=0
    #    labels[newdataIndex[np.where(newlabels==1)[0]]]=1
    #labels=np.uint8(255-np.reshape(labels,(Img.shape[1],Img.shape[0])).T*51)
    #labels[np.where(labels!=204)]=0
    #labels[np.where(labels==204)]=255
    return labels

def caculateShelterRate(center,radius,img,count,lor):
    print '-------'
    print 'caculateShelterRate'
    countStr='%d'%count
    numStr='%d'%lor
    topShelter=[]
    bottomShelter=[]
    radius=int(round(radius))
    topedge=center[1]-radius
    bottomedge=center[1]+radius
    print img.shape
    print 'radius=%d'%(radius)
    print 'topedge=%d'%(topedge)
    print 'bottomedge=%d'%(bottomedge)
    print 'img.shape[0]-1'
    print img.shape[0]-1
    if topedge<0 or bottomedge>=img.shape[0]-1:
        topShelter.append(np.int32(irisRange[len(irisRange)/2]-2))
        bottomShelter.append(2)
        print 'return 0'
        return 0,topShelter,bottomShelter
    sumData=np.int64(np.sum(img[:,center[0]-4:center[0]+4],axis=1))
    print 'sumData'
    print sumData
    diffData=np.diff(sumData)
    print '-'*40
    print diffData
    topPos=np.where(diffData[topedge-5:center[1]]==max(diffData[topedge-5:center[1]]))[0][-1]+topedge-5
    bottomPos=np.where(diffData[center[1]:bottomedge+5]==min(diffData[center[1]:bottomedge+5]))[0][0]+center[1]
    if topPos<0:
        topPos=0
    if topPos>=center[1]:
        topPos=center[1]-1
    if bottomPos>img.shape[0]-1:
        bottomPos=img.shape[0]-1
    if bottomPos<=center[1]:
        bottomPos=center[1]+1
    
    temp2Img=cp.deepcopy(img)
    temp2Img[topPos:bottomPos+1,center[0]]=100
    #cv2.imshow('temp2Img_%d_%d'%(count,lor),temp2Img)
    #cv2.waitKey()
    
    out=cp.deepcopy(img[:,center[0]])
    out[topedge]=111
    out[bottomedge+1]=111
    tempImg=cp.deepcopy(img)
    tempImg[topedge:bottomedge+1,center[0]]=100
    #cv2.imshow('img_%d_%d'%(count,lor),tempImg)
    #cv2.waitKey()
    print '-'*40
    print out
    print topPos
    print bottomPos
    topShelter.append(topPos-topedge+1)
    #topShelter.append(topPos-topedge+2)
    bottomShelter.append(bottomedge-bottomPos+1)
    #bottomShelter.append(bottomedge-bottomPos+2)
    print 'topShelter'
    print topShelter
    print 'bottomShelter'
    print bottomShelter
    
    img[topPos:bottomPos+1,center[0]]=150
    img[center[1],center[0]-radius:center[0]+radius]=150
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/kmeans/eye'+countStr+'_'+numStr+'.jpg',img)
    print 'return 1'
    return 1,topShelter,bottomShelter

In [ ]:
#3.1模板选择
def control(leftEyeRectified,rightEyeRectified,frameCount):
    print '-------'
    print 'control'
    information=[]
    global leftEyeInfo
    global rightEyeInfo
    eyecenterInRect=[]
    IrisRadius=[]
    diff=[]
    DaugmanCenter=[]
    DaugmanRadius=[]
    EyeFiltered=[]

    leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = DaugmanIrisLocalization(leftEyeRectified,frameCount,1)
    reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = DaugmanIrisLocalization(rightEyeRectified,frameCount,2)
    
    eyecenterInRect.append(leyecenterInRect)
    eyecenterInRect.append(reyecenterInRect)
    IrisRadius.append(lIrisRadius)
    IrisRadius.append(rIrisRadius)
    diff.append(diffLeft)
    diff.append(diffRight)
    DaugmanCenter.append(leftDaugmanCenter)
    DaugmanCenter.append(rightDaugmanCenter)
    DaugmanRadius.append(leftDaugmanRadius)
    DaugmanRadius.append(rightDaugmanRadius)
    EyeFiltered.append(leftEyeFiltered)
    EyeFiltered.append(rightEyeFiltered)
    return 1,eyecenterInRect,IrisRadius,diff,DaugmanCenter,DaugmanRadius,EyeFiltered

In [ ]:
def DaugmanIrisLocalization(img,count,lor): 
    print 'irisRange:'
    print irisRange
    
    countStr='%d'%count
    numStr='%d'%lor
    
    fillhole_and_kmeans=0
    
    imggrey,imgfiltered=preprocessing(img,count,lor)
    
    #fillhole_and_kmeans_start
    if fillhole_and_kmeans==1:
        ret,imgBinary=cv2.threshold(imggrey,70,255,cv2.THRESH_BINARY)
        imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
        cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/fillhole/eye'+countStr+'_'+numStr+'.jpg',imgNoHole)
        imgKmeans=UseKmeans(imgNoHole,imggrey,count,lor)
        temp=imgKmeans/255
    #fillhole_and_kmeans_end
    
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]
    parallelResults = [r.get() for r in results]        
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2])    
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    
    #debug_start
    tempcovstackdiffmaxsumfiltered=cp.deepcopy(covstackdiffmaxsumfiltered)#float64
    tempcovstackdiffmaxsumfiltered=tempcovstackdiffmaxsumfiltered/np.max(np.max(tempcovstackdiffmaxsumfiltered))
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/max/max'+countStr+'_'+numStr+'.jpg',tempcovstackdiffmaxsumfiltered*255)
    #debug_end
    
    #add_weight_start
    #covstackdiffmaxsumfiltered=covstackdiffmaxsumfiltered*imggrey
    #add_weight_end
    
    #fillhole_and_kmeans_start
    if fillhole_and_kmeans==1:
        covstackdiffmaxsumfiltered=covstackdiffmaxsumfiltered*temp
    #fillhole_and_kmeans_end
        
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    DaugmanIrisCenter=list(DaugmanIrisCenter)
    DaugmanIrisCenter[0]=DaugmanIrisCenter[0]+1
    DaugmanIrisCenter=tuple(DaugmanIrisCenter)
    DaugmanIrisRadius=DaugmanIrisRadius-1
    global irisRadiiLog
    irisRadiiLog.append(DaugmanIrisRadius/imageWorkingWidth)
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    
    cv2.circle(imgfiltered,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),1,(255,0,0))
    cv2.circle(imgfiltered,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),np.int32(DaugmanIrisRadius),(255,0,0))
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/center/eye'+countStr+'_'+numStr+'.jpg',cv2.cvtColor(imgfiltered,cv2.COLOR_RGB2BGR))
    
    print 'DaugmanIrisCenter'
    print DaugmanIrisCenter
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    
    
    #countStr='%d'%count
    #numStr='%d'%lor
    #ret,imgBinary=cv2.threshold(imggrey,150,255,cv2.THRESH_BINARY)
    #imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
    #cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/fillhole/eye'+countStr+'_'+numStr+'.jpg',imgNoHole)
    #imgKmeans=UseKmeans(imgNoHole,imggrey,count,lor)
    #cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/kmeans/eye'+countStr+'_'+numStr+'.jpg',imgKmeans)
    
    
    return eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered

In [ ]:
def DaugmanIrisLocalization1(img,count,lor):
    print '-'
    countStr='%d'%count
    numStr='%d'%lor
    print 'DaugmanIrisLocalization1'
    if lor==1:
        print '左眼'
    else:
        print '右眼'
        
    imggrey,imgfiltered=preprocessing(img,count,lor)
    
    info=[]
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(irisTemplateSet)])  
    
    results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in irisTemplateSet]    
    parallelResults = [r.get() for r in results]    
    for s in xrange(len(parallelResults)):
        covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
        covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
        covstackDiffSet.append(parallelResults[s][2]) 
        
    covstackdiffmaxsum = 0
    for s in xrange(len(irisTemplateSet)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeights[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    
    DaugmanIrisCenter=list(DaugmanIrisCenter)
    DaugmanIrisCenter[0]=DaugmanIrisCenter[0]+1
    DaugmanIrisCenter=tuple(DaugmanIrisCenter)
    DaugmanIrisRadius=DaugmanIrisRadius-1
    
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    
    
    ret,imgBinary=cv2.threshold(imggrey,150,255,cv2.THRESH_BINARY)
    imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/fillhole/eye'+countStr+'_'+numStr+'.jpg',imgNoHole)
    imgKmeans=UseKmeans(imgNoHole,imggrey,count,lor)
    #if imgKmeans[np.where(imgKmeans==255)].shape[0]<1200:
    #    return 0,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info
    suc,topShelterRate,bottomShelterRate=caculateShelterRate(DaugmanIrisCenter,DaugmanIrisRadius,imgKmeans,count,lor)   
    if suc==0:
        info.append(np.int32(irisRange[len(irisRange)/2]))
    else:
        info.append(DaugmanIrisRadius)
    info.append(topShelterRate)
    info.append(bottomShelterRate)
    info.append(DaugmanIrisCenter)
    
    return 1,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info

In [ ]:
def DaugmanIrisLocalization2(img,info,count,lor):
    print 'DaugmanIrisLocalization2'
    if lor==1:
        print '左眼'
    else:
        print '右眼'
    print 'irisRange2'
    print irisRange2
    print 'irisRange'
    print irisRange
    
    countStr='%d'%count
    numStr='%d'%lor
    print 'info'
    print info
    imggrey,imgfiltered=preprocessing(img,count,lor)
    templateIndex2=info[1]
    print templateIndex2
    templateIndex3=info[2]
    print templateIndex3
    myTemp=[]
    irisWeightsThis=[]
    for i in xrange(0,len(templateIndex2)):
        for j in xrange(0,len(templateIndex3)):
            mytemplate=[]
            for k in irisRange:
                deviation=np.int32(k-info[0])
                #对于超过模板上下边界的情况，用边界值：例如上半边最小截取0，最大截取半径
                index1=np.int32(k)-np.int32(irisRange2[0])
                index2=templateIndex2[i]+deviation
                index3=templateIndex3[j]+deviation         
                if index2<0:
                    index2=0
                if index2>=len(irisTemplateSet2[index1]):
                    index2=len(irisTemplateSet2[index1])-1
                if index3<0:
                    index3=0
                if index3>=len(irisTemplateSet2[index1][index2]):
                    index3=len(irisTemplateSet2[index1][index2])-1
                 
                print 'i=%d,j=%d,k=%d'%(i,j,k)
                print 'index1=%d,index2=%d,index3=%d'%(index1,index2,index3)
                
                temp=irisTemplateSet2[index1][index2][index3]
                mytemplate.append(temp)
                temp=temp*255
                temp=np.int8(temp)
                #cv2.imwrite('/Users/ly/Code/templates/eye'+countStr+'_'+numStr+'_'+k+'.jpg',temp)
                cv2.imshow('temp',np.int8(temp*255))
                #cv2.waitKey()
            myTemp.append(mytemplate)
            irisWeightsThis.append(1)
    
    
    ret,imgBinary=cv2.threshold(imggrey,70,255,cv2.THRESH_BINARY)
    imgNoHole=fillHoleInBinary(imgfiltered,imgBinary)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/fillhole/eye'+countStr+'_'+numStr+'.jpg',imgNoHole)
    imgKmeans=UseKmeans(imgNoHole,imggrey,count,lor)
    
    
    
    infoNew=[]
    covstackDiffSet = []
    covstackdiffmaxmaps = np.zeros(list(imggrey.shape) + [len(myTemp)])
    covstackdiffmaxradii = np.zeros(list(imggrey.shape) + [len(myTemp)])

    #results = [mypool.apply_async(DaugmanIrisCore.DaugmanIrisCore,(irisTemplates,imggrey)) for irisTemplates in myTemp]
    #parallelResults = [r.get() for r in results]

    #for s in xrange(len(parallelResults)):
    #    covstackdiffmaxmaps[:,:,s] = parallelResults[s][0]
    #    covstackdiffmaxradii[:,:,s] = parallelResults[s][1]
    #    covstackDiffSet.append(parallelResults[s][2])
        
    
    parallelResults=DaugmanIrisCore.DaugmanIrisCore2(myTemp[0],imggrey,countStr,numStr)
    covstackdiffmaxmaps[:,:,0] = parallelResults[0]
    covstackdiffmaxradii[:,:,0] = parallelResults[1]
    covstackDiffSet.append(parallelResults[2])
    
    covstackdiffmaxsum = 0
    for s in xrange(len(myTemp)):
        covstackdiffmaxsum += covstackdiffmaxmaps[:,:,s]*irisWeightsThis[s]
    covstackdiffmaxsum **= 3
    covstackdiffmaxsumfiltered = scipy.ndimage.gaussian_filter(covstackdiffmaxsum,6,0,None,mode='nearest')
    
    #DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    #挑选在空白区域最大的值
    temp=imgKmeans/255
    #DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    x=np.int32(np.float64(sum(np.where(temp==1)[0]))/np.float64(sum(sum(temp))))
    y=np.int32(np.float64(sum(np.where(temp==1)[1]))/np.float64(sum(sum(temp))))
    print 'x,y'
    print x
    print y
    ring=np.zeros(temp.shape,dtype=np.uint8)
    cv2.circle(ring,(x,y),np.int32(irisRange[len(irisRange)-1]/2),1,-1)
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/ring/eye'+countStr+'_'+numStr+'.jpg',ring*255)
    print 'ring'
    print ring
    print ring.shape
    print covstackdiffmaxsumfiltered.shape
    print sum(sum(ring))
    tempnew=temp*ring
    if sum(sum(tempnew))==0:
        covstackdiffmaxsumfiltered=temp*covstackdiffmaxsumfiltered
    else:
        covstackdiffmaxsumfiltered=tempnew*covstackdiffmaxsumfiltered
    DaugmanIrisCenter = np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    print 'DaugmanIrisCenter'
    print DaugmanIrisCenter
    
    
    #挑选和图中心最接近的
    #start=20
    #i=start
    #if i>0:    
    #    pos=np.unravel_index(np.argmax(covstackdiffmaxsumfiltered),covstackdiffmaxsumfiltered.shape)
    #    if i==start:
    #        mindis=(pos[0]-covstackdiffmaxsumfiltered.shape[0]/2)**2+(pos[1]-covstackdiffmaxsumfiltered.shape[1]/2)**2
    #        DaugmanIrisCenter=pos
    #        covstackdiffmaxsumfiltered[DaugmanIrisCenter[0],DaugmanIrisCenter[1]]=0
    #    else:
    #        if (pos[0]-covstackdiffmaxsumfiltered.shape[0]/2)**2+(pos[1]-covstackdiffmaxsumfiltered.shape[1]/2)**2<mindis:
    #            mindis=(pos[0]-covstackdiffmaxsumfiltered.shape[0]/2)**2+(pos[1]-covstackdiffmaxsumfiltered.shape[1]/2)**2
    #            DaugmanIrisCenter=pos
    #            covstackdiffmaxsumfiltered[DaugmanIrisCenter[0],DaugmanIrisCenter[1]]=0
    #    i=i-1
        
    DaugmanIrisRadius = irisRange[np.round(scipy.stats.nanmedian(covstackdiffmaxradii[DaugmanIrisCenter[0],DaugmanIrisCenter[1],:]))]
    DaugmanIrisCenter = DaugmanIrisCenter[::-1]
    
    cv2.circle(imgfiltered,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),1,(0,0,255))
    cv2.circle(imgfiltered,tuple(np.int32(np.squeeze(DaugmanIrisCenter))),np.int32(DaugmanIrisRadius),(0,0,255))
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/center/eye'+countStr+'_'+numStr+'.jpg',imgfiltered)

    
    eyecenter = np.float64(DaugmanIrisCenter) * (img.shape[1]/imageWorkingWidth)
    eyeiris =   np.float64(DaugmanIrisRadius) * (img.shape[1]/imageWorkingWidth)
    
    #进行约束，如果得到的圆心和上一帧的圆心的距离大于阈值，那么就使用上一帧的圆心和半径
    #thresholdDistanceCenter=np.float64(info[0])
    #if np.sqrt(np.float64((DaugmanIrisCenter[0]-info[3][0])**2+(DaugmanIrisCenter[1]-info[3][1])**2))>thresholdDistanceCenter:
    #    DaugmanIrisCenter=list(DaugmanIrisCenter)
    #    DaugmanIrisCenter=info[3]
    #    DaugmanIrisCenter=tuple(DaugmanIrisCenter)
    #    DaugmanIrisRadius=info[0]

    

    #if imgKmeans[np.where(imgKmeans==255)].shape[0]<1200:
    #    return 0,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,info
    suc,topShelterRate,bottomShelterRate=caculateShelterRate(DaugmanIrisCenter,DaugmanIrisRadius,imgKmeans,count,lor)
    if suc==0:
        info.append(np.int32(irisRange[len(irisRange)/2]))
    else:
        info.append(DaugmanIrisRadius)
    infoNew.append(DaugmanIrisRadius)
    infoNew.append(topShelterRate)
    infoNew.append(bottomShelterRate)
    infoNew.append(DaugmanIrisCenter)
    
    return 1,eyecenter,eyeiris,covstackdiffmaxsum,DaugmanIrisCenter,DaugmanIrisRadius,imgfiltered,infoNew